In [3]:
import requests
import pandas as pd
import dotenv
import os

dotenv.load_dotenv()

# Define the headers, including any necessary authentication
headers = {
    "Content-Type": "application/json",
    "ApiKey": os.getenv("ApiKey"),
}


In [2]:
# Define the GraphQL query
query = """
query Scenes($input: SceneQueryInput!) {
    queryScenes(input: $input) {
        count
        scenes {
            id
        }
    }
}
"""

variables = {
    "input": {
        "page": 1,
        "per_page": 20,
        "sort": "DATE",
        "direction": "DESC",
        "parentStudio": "324ea274-1afb-4f80-aa66-c6ddb52b9b56"
    }
}

# Make the POST request
response = requests.post('https://stashdb.org/graphql', json={'query': query, 'variables': variables}, headers=headers)

# Parse the JSON response into a DataFrame
data = response.json()
df = pd.json_normalize(data['data'])  # Use pd.json_normalize to flatten the data if it's nested

In [3]:
# Read the GraphQL query from a file
with open('stashdb-graphql/findSceneByFingerprint.graphql', 'r') as file:
    query = file.read()

# Execute the query as before
response = requests.post('https://stashdb.org/graphql', json={'query': query, 'variables': variables}, headers=headers)

# Parse the JSON response into a DataFrame
data = response.json()
single_scene_df = pd.json_normalize(data['data']['findSceneByFingerprint'])  # Use pd.json_normalize to flatten the data if it's nested


In [4]:
# Read the GraphQL query from a file
with open('stashdb-graphql/studio_scenes.graphql', 'r') as file:
    query = file.read()

variables = {
    "input": {
        "page": 1,
        "per_page": 20,
        "sort": "DATE",
        "direction": "DESC",
        "parentStudio": "324ea274-1afb-4f80-aa66-c6ddb52b9b56"
    }
}

# Execute the query as before
response = requests.post('https://stashdb.org/graphql', json={'query': query, 'variables': variables}, headers=headers)

# Parse the JSON response into a DataFrame
data = response.json()
studio_scenes_df = pd.json_normalize(data['data']['queryScenes']['scenes'])  # Use pd.json_normalize to flatten the data if it's nested


In [5]:
# Read the GraphQL query from a file
with open('stashdb-graphql/studio_scenes.graphql', 'r') as file:
    query = file.read()

# Initialize variables
page = 1
all_scenes = []
total_scenes = None

while total_scenes is None or len(all_scenes) < total_scenes:
    variables = {
        "input": {
            "page": page,
            "per_page": 20,
            "sort": "DATE",
            "direction": "DESC",
            "parentStudio": "324ea274-1afb-4f80-aa66-c6ddb52b9b56"
        }
    }

    # Execute the query
    response = requests.post('https://stashdb.org/graphql', json={'query': query, 'variables': variables}, headers=headers)
    data = response.json()

    # Append the scenes to all_scenes
    all_scenes.extend(data['data']['queryScenes']['scenes'])

    # Get the total number of scenes if it's not already set
    if total_scenes is None:
        total_scenes = data['data']['queryScenes']['count']

    # Increment the page number
    page += 1

# Convert all_scenes to a DataFrame
studio_scenes_df = pd.json_normalize(all_scenes)  # Use pd.json_normalize to flatten the data if it's nested